In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
import DifferentialEquations as DE
include("multiharmonic_balance.jl");

In [ ]:
gamma = 0;
omega = 5;
gamma3 = 40;
g0 = 9.80665; # m / s^2
height = 5; # m

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
yleft = 0.0;
yright = 1.0;
Nt = 5
Nx = Ny = 15;
harmonics = 3; # number of harmonics
order = 2;
stepx = (xright-xleft)/Nx;
stepy = (yright - yleft)/Ny;

In [ ]:
# Define symbolics
Model.@parameters x, y, t;

Dy = Model.Differential(y)
Dx = Model.Differential(x);
Dt = Model.Differential(t);

In [ ]:
u0 = ones((Nx + 1) * (Ny + 1) * harmonics * 2)*0.05;

In [ ]:
vars, var_exprs, u = create_ansatz((x, y), t, omega, harmonics);
bcs = create_bcs(vars, ((xleft, xright), (yleft, yright)), (x, y), 0.0)

F = 50 * exp(-40*(x^2))*sin(omega*t)

pde = Dt(Dt(u)) - 9*(Dx(Dx(u)) + Dy(Dy(u))) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;

Model.@variables x, y, t;
expanded = expand_trig(pde, t, omega);
eqs = make_residual(expanded, harmonics, omega, t);
eqs

In [ ]:
solution_coeffs = solve_harmonicbalance(
                        eqs, harmonics, bcs,
                        ((xleft, xright), (yleft, yright)),
                        (stepx, stepy),
                        (x, y),
                        var_exprs, u0
                    )

u0 = vcat(solution_coeffs...);

In [ ]:
using Plots
anim = @animate for t in 1:0.1:10
    if t % 100 == 0
        println(t)
    end

    u_new = solution_coeffs[1]*0.0
    j = 1
    for i in 1:(2*harmonics)
        if isodd(i)
            u_new .+= solution_coeffs[i] .* sin(j * omega * t)
        else
            u_new .+= solution_coeffs[i] .* cos(j * omega * t)
            j += 1
        end
    end
    

    heatmap(u_new, clims=(-0.1, 0.1))
    title!("(Solvetime = 296s) Wave Equation with H = $(harmonics), \$\\gamma\$ = $(gamma), \$\\gamma_3\$ = $(gamma3), \$\\omega\$ = $omega", titlefontsize=10)

end

gif(anim, "HB_WE_2D.gif", fps=30)

In [ ]:
# runtime for: omega = 3, gamma3 = 40, harmonics = 1, gamma = 0,c^2 = 9. Runtime: 296s